In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from scipy.stats import ttest_ind
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import classification_report, roc_auc_score
import xgboost as xgb


In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# 1) Load concat features and metadata
data = np.load("./data/doc_features/transcript_componenttext_2012_2_features.npz", allow_pickle=True)

Xc = data["X_concat"]        # shape (N_docs, 2*D)
tids = data["transcriptids"]  # same order
meta = pd.read_csv("./data/doc_features/transcript_componenttext_2012_2_features_meta.csv")

meta_unique = (
    meta[["transcriptid", "SUESCORE", "label"]]
    .drop_duplicates(subset="transcriptid", keep="first")
    .set_index("transcriptid")
)

# 3) build a mask over tids
mask = np.isin(tids, meta_unique.index)

Xc = Xc[mask]
tids = tids[mask]

# 2) Build labels and mask
meta = meta.assign(label=lambda df: df.SUESCORE.map(lambda s: 1 if s>=0.5 else (0 if s<=-0.5 else np.nan)))
mask = meta.label.notna().values
X_test_all_feat, y_test = Xc[mask], meta.loc[mask, "label"].astype(int).values

In [3]:
# 1) Load concat features and metadata
data = np.load("./data/doc_features/transcript_componenttext_2012_1_features.npz", allow_pickle=True)

Xc = data["X_concat"]        # shape (N_docs, 2*D)
tids = data["transcriptids"]  # same order
meta = pd.read_csv("./data/doc_features/transcript_componenttext_2012_1_features_meta.csv")

meta_unique = (
    meta[["transcriptid", "SUESCORE", "label"]]
    .drop_duplicates(subset="transcriptid", keep="first")
    .set_index("transcriptid")
)

# 3) build a mask over tids
mask = np.isin(tids, meta_unique.index)

Xc = Xc[mask]
tids = tids[mask]

# 2) Build labels and mask
meta = meta.assign(label=lambda df: df.SUESCORE.map(lambda s: 1 if s>=0.5 else (0 if s<=-0.5 else np.nan)))
mask = meta.label.notna().values
X_val_all_feat, y_val = Xc[mask], meta.loc[mask, "label"].astype(int).values

In [4]:
npz_paths = [
    "./data/doc_features/transcript_componenttext_2010_1_features.npz",
    "./data/doc_features/transcript_componenttext_2010_2_features.npz",
    "./data/doc_features/transcript_componenttext_2011_1_features.npz",
    "./data/doc_features/transcript_componenttext_2011_2_features.npz",
    # "./data/doc_features/transcript_componenttext_2012_1_features.npz",
]

In [5]:
X_list = []
y_list = []

for npz_path in npz_paths:
 
    base = os.path.splitext(os.path.basename(npz_path))[0]      
    csv_path = os.path.join(
        os.path.dirname(npz_path),
        base + "_meta.csv"                                       
    )


    data = np.load(npz_path, allow_pickle=True)
    # X_concat = data["X_concat"]      # (N_docs, 2*D)
    X_concat = data["X_concat"]  
    tids = data["transcriptids"]    


    meta = pd.read_csv(csv_path)

    meta_unique = (
        meta[["transcriptid", "SUESCORE", "label"]]
        .drop_duplicates(subset="transcriptid", keep="first")
        .set_index("transcriptid")
    )

    mask_ids = np.isin(tids, meta_unique.index)
    X_filt = X_concat[mask_ids]
    tids_filt = np.array(tids)[mask_ids]


    lab_df = meta.assign(
        label=lambda df: df.SUESCORE.map(
            lambda s: 1 if s >= 0.5 else (0 if s <= -0.5 else np.nan)
        )
    )
    mask_label = lab_df.label.notna().values
    # apply the same mask in the same order as the CSV, so we use .loc on lab_df
    # but first filter lab_df to only those transcriptids in tids_filt
    Xc, y = X_filt[mask_label], meta.loc[mask_label, "label"].astype(int).values
    
    # now align X and y
    # X_final = X_filt[lab_sub.label.notna()]
    # y_final = lab_sub.label.astype(int).values

    # collect
    X_list.append(Xc)
    y_list.append(y)

# 2. concatenate all files together
Xc = np.vstack(X_list)   # shape: (sum_i N_i, 2*D)
y  = np.concatenate(y_list)  # shape: (sum_i N_i,)

print("Combined Xc shape:", Xc.shape)
print("Combined y shape: ", y.shape)

Combined Xc shape: (3561, 32768)
Combined y shape:  (3561,)


In [6]:
# forced resampling
idx0 = np.where(y == 0)[0]
idx1 = np.where(y == 1)[0]

n = min(len(idx0), len(idx1))

sel0 = np.random.choice(idx0, size=n, replace=False)
sel1 = np.random.choice(idx1, size=n, replace=False)

sel = np.concatenate([sel0, sel1])
np.random.shuffle(sel)

# slice out your balanced subset
Xc = Xc[sel]
y = y[sel]

print("Balanced X shape:", Xc.shape)
print("Balanced y counts:", np.bincount(y))

Balanced X shape: (1450, 32768)
Balanced y counts: [725 725]


In [13]:


# meta = meta.assign(label=lambda df: df.SUESCORE.map(lambda s: 1 if s>=0.5 else (0 if s<=-0.5 else np.nan)))
# mask = meta.label.notna().values
# Xc, y = Xc_aligned[mask], meta.loc[mask, "label"].astype(int).values

D2 = Xc.shape[1]
D = D2 // 2
X_pos, X_neg = Xc[y==1], Xc[y==0]
t_stats = np.abs((X_pos.mean(0) - X_neg.mean(0)) /
                 np.sqrt(X_pos.var(0)/len(X_pos) + X_neg.var(0)/len(X_neg)))

ranked_idx = np.argsort(-t_stats)


for rank, idx in enumerate(ranked_idx[:1000], start=1):
    # print(idx)
    part = "mean" if idx < D else "max"
    # print(idx)
    # print(D)
    feat_id = idx if idx < D else idx-D
    t_val   = t_stats[idx]
    print(f"Rank {rank:2d}: {part!r} feature #{feat_id} (t = {t_val:.2f})")



Rank  1: 'max' feature #3081 (t = 9.02)
Rank  2: 'max' feature #6700 (t = 8.74)
Rank  3: 'mean' feature #4983 (t = 8.61)
Rank  4: 'max' feature #188 (t = 8.60)
Rank  5: 'max' feature #7429 (t = 8.55)
Rank  6: 'max' feature #6893 (t = 8.42)
Rank  7: 'max' feature #3581 (t = 8.32)
Rank  8: 'max' feature #2867 (t = 8.24)
Rank  9: 'max' feature #957 (t = 8.18)
Rank 10: 'mean' feature #1881 (t = 8.14)
Rank 11: 'max' feature #2445 (t = 8.13)
Rank 12: 'max' feature #4808 (t = 8.07)
Rank 13: 'mean' feature #7456 (t = 8.01)
Rank 14: 'max' feature #1818 (t = 7.92)
Rank 15: 'max' feature #7546 (t = 7.92)
Rank 16: 'mean' feature #1211 (t = 7.87)
Rank 17: 'max' feature #5888 (t = 7.86)
Rank 18: 'mean' feature #6902 (t = 7.79)
Rank 19: 'mean' feature #3878 (t = 7.74)
Rank 20: 'max' feature #4107 (t = 7.71)
Rank 21: 'max' feature #1963 (t = 7.60)
Rank 22: 'mean' feature #7357 (t = 7.57)
Rank 23: 'max' feature #8086 (t = 7.54)
Rank 24: 'max' feature #5425 (t = 7.46)
Rank 25: 'mean' feature #1547 (t = 

/tmp/ipykernel_415405/4004282485.py:8: RuntimeWarning: invalid value encountered in divide
  t_stats = np.abs((X_pos.mean(0) - X_neg.mean(0)) /


In [12]:
# top_idx = ranked_idx[:]
# X_test = X_test_all_feat[:, :16384]
# X_val = X_val_all_feat[:, :16384]
# X_top = Xc[:, :16384]      
X_test = X_test_all_feat[:, 16384:]
X_val = X_val_all_feat[:, 16384:]
X_top = Xc[:, 16384:]   

X_train = X_top
y_train = y


In [ ]:
# 5) Train with L1 logistic regression & balanced class weights
clf = make_pipeline(
    StandardScaler(),
    LogisticRegression(
        penalty="l1",
        solver="saga",
        # class_weight="balanced",
        C=1.0,
        max_iter=2000,
        random_state=42
    )
)
clf.fit(X_train, y_train)
clf.fit(X_train, y_train)

# 6) Evaluate
y_pred   = clf.predict(X_test)
y_probs  = clf.predict_proba(X_test)[:,1]

print(classification_report(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_probs))

# 7) Inspect which of your top-1000 actually got nonzero weights
lr = clf.named_steps["logisticregression"]
coefs = lr.coef_.ravel()
nz    = np.where(coefs != 0)[0]



In [10]:

param_grid = {"logisticregression__C": [0.01, 0.1, 1, 10, 100]}

pipeline = make_pipeline(
    StandardScaler(),
    LogisticRegression(
        penalty="l2",
        solver="saga",    
        # solver="liblinear",    
        # class_weight="balanced",
        max_iter=10000,
        random_state=42
    )
)

search = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,
    scoring="roc_auc",
    n_jobs=-1,
    verbose=1
)
search.fit(X_train, y_train)

print("Best C (inverse reg. strength):", search.best_params_["logisticregression__C"])
print("CV ROC AUC:", search.best_score_)


best_clf = search.best_estimator_
y_pred_probs = best_clf.predict_proba(X_test)[:, 1]
y_pred       = best_clf.predict(X_test)

print(classification_report(y_test, y_pred))
print("Test ROC AUC:", roc_auc_score(y_test, y_pred_probs))

Fitting 5 folds for each of 5 candidates, totalling 25 fits


: 

In [ ]:

X_train_t = torch.from_numpy(X_train).float().to(device)
y_train_t = torch.from_numpy(y_train).float().unsqueeze(1).to(device)
X_test_t  = torch.from_numpy(X_test).float().to(device)
y_test_t  = torch.from_numpy(y_test).float().unsqueeze(1).to(device)

# DataLoader
train_ds = TensorDataset(X_train_t, y_train_t)
train_dl = DataLoader(train_ds, batch_size=32, shuffle=True, drop_last=True)


class ShallowMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim=256, dropout=0.5):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid()
        )
    def forward(self, x):
        return self.net(x)

model = ShallowMLP(input_dim=X_top.shape[1]).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-5)
criterion = nn.BCELoss()

# 4) Training loop
n_epochs = 20
for epoch in range(1, n_epochs+1):
    model.train()
    total_loss = 0.0
    for xb, yb in train_dl:
        pred = model(xb)
        loss = criterion(pred, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * xb.size(0)
    avg_loss = total_loss / len(train_dl.dataset)
    print(f"Epoch {epoch:2d}: train loss = {avg_loss:.4f}")

# 5) Evaluation
model.eval()
with torch.no_grad():
    y_prob = model(X_test_t).cpu().numpy().flatten()
    y_pred = (y_prob >= 0.5).astype(int)

print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("Test ROC AUC:", roc_auc_score(y_test, y_prob))

Epoch  1: train loss = 0.5225
Epoch  2: train loss = 0.3806
Epoch  3: train loss = 0.2680
Epoch  4: train loss = 0.1722
Epoch  5: train loss = 0.1100
Epoch  6: train loss = 0.0770
Epoch  7: train loss = 0.1001
Epoch  8: train loss = 0.0498
Epoch  9: train loss = 0.0583
Epoch 10: train loss = 0.0798
Epoch 11: train loss = 0.0553
Epoch 12: train loss = 0.0487
Epoch 13: train loss = 0.0444
Epoch 14: train loss = 0.0385
Epoch 15: train loss = 0.0406
Epoch 16: train loss = 0.0573
Epoch 17: train loss = 0.0739
Epoch 18: train loss = 0.0661
Epoch 19: train loss = 0.0637
Epoch 20: train loss = 0.0357

Classification Report:
              precision    recall  f1-score   support

           0       0.31      0.29      0.30       222
           1       0.80      0.82      0.81       793

    accuracy                           0.71      1015
   macro avg       0.56      0.56      0.56      1015
weighted avg       0.70      0.71      0.70      1015

Test ROC AUC: 0.603447394431001


In [7]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest  = xgb.DMatrix(X_test,  label=y_test)


scale_pos_weight = float((y_train == 0).sum()) / (y_train == 1).sum()


params = {
    "objective":        "binary:logistic",
    "eval_metric":      "auc",
    "scale_pos_weight": scale_pos_weight,
    "tree_method":      "hist",       
    "grow_policy":      "lossguide",  
    "max_depth":        6,
    "learning_rate":    0.1,
    "subsample":        0.8,
    "colsample_bytree": 0.8,
    "random_state":     42,
    "verbosity":        1
}


cv_results = xgb.cv(
    params,
    dtrain,
    num_boost_round=1000,
    nfold=5,
    early_stopping_rounds=20,
    metrics="auc",
    seed=42,
    as_pandas=True,
    verbose_eval=50
)
best_rounds = len(cv_results)
print(f"Optimal boosting rounds: {best_rounds}")


bst = xgb.train(
    params,
    dtrain,
    num_boost_round=best_rounds
)


y_prob = bst.predict(dtest)
y_pred = (y_prob >= 0.5).astype(int)

print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("Test ROC AUC:", roc_auc_score(y_test, y_prob))

: 

In [17]:
# 3 layer mlp

def to_tensor_dataset(X, y):
    Xt = torch.from_numpy(X).float().to(device)
    yt = torch.from_numpy(y).float().unsqueeze(1).to(device)
    return TensorDataset(Xt, yt)

train_ds = to_tensor_dataset(X_train, y_train)
val_ds   = to_tensor_dataset(X_val, y_val)
test_ds  = to_tensor_dataset(X_test, y_test)

batch_size = 32
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True, drop_last=True)
val_dl   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False)
test_dl  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False)

# 3) Define the 3-layer MLP
class ThreeLayerMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim=256, dropout=0.5):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),

            nn.Linear(hidden_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),

            nn.Linear(hidden_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),

            nn.Linear(hidden_dim, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)

model = ThreeLayerMLP(input_dim=X_train.shape[1]).to(device)

# 4) Optimizer and loss
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-6, weight_decay=1e-5)
criterion = nn.BCELoss()

# 5) Training & Validation Loop
n_epochs = 300
best_val_loss = float('inf')

for epoch in range(1, n_epochs+1):
    # -- Training
    model.train()
    train_loss = 0.0
    for xb, yb in train_dl:
        preds = model(xb)
        loss = criterion(preds, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * xb.size(0)
    train_loss /= len(train_dl.dataset)

    # -- Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for xb, yb in val_dl:
            preds = model(xb)
            loss = criterion(preds, yb)
            val_loss += loss.item() * xb.size(0)
    val_loss /= len(val_dl.dataset)

    print(f"Epoch {epoch:2d}  Train Loss: {train_loss:.4f}  Val Loss: {val_loss:.4f}")

    # Optional: save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "best_model.pth")

# 6) Load best model and test evaluation
model.load_state_dict(torch.load("best_model.pth"))
model.eval()

y_probs = []
y_true  = []
with torch.no_grad():
    for xb, yb in test_dl:
        probs = model(xb)
        y_probs.extend(probs.cpu().numpy().flatten().tolist())
        y_true .extend(yb.cpu().numpy().flatten().tolist())

y_pred = (np.array(y_probs) >= 0.5).astype(int)

print("\nTest Classification Report:")
print(classification_report(y_true, y_pred))
print("Test ROC AUC:", roc_auc_score(y_true, y_probs))


Epoch  1  Train Loss: 0.7997  Val Loss: 0.7110
Epoch  2  Train Loss: 0.8021  Val Loss: 0.7170
Epoch  3  Train Loss: 0.7923  Val Loss: 0.7106
Epoch  4  Train Loss: 0.7861  Val Loss: 0.7077
Epoch  5  Train Loss: 0.7820  Val Loss: 0.7075
Epoch  6  Train Loss: 0.7804  Val Loss: 0.7122
Epoch  7  Train Loss: 0.7810  Val Loss: 0.7059
Epoch  8  Train Loss: 0.7785  Val Loss: 0.6966
Epoch  9  Train Loss: 0.7692  Val Loss: 0.7002
Epoch 10  Train Loss: 0.7636  Val Loss: 0.6978
Epoch 11  Train Loss: 0.7578  Val Loss: 0.6964
Epoch 12  Train Loss: 0.7569  Val Loss: 0.6925
Epoch 13  Train Loss: 0.7509  Val Loss: 0.6911
Epoch 14  Train Loss: 0.7407  Val Loss: 0.6904
Epoch 15  Train Loss: 0.7380  Val Loss: 0.6897
Epoch 16  Train Loss: 0.7382  Val Loss: 0.6918
Epoch 17  Train Loss: 0.7306  Val Loss: 0.6853
Epoch 18  Train Loss: 0.7281  Val Loss: 0.6788
Epoch 19  Train Loss: 0.7247  Val Loss: 0.6831
Epoch 20  Train Loss: 0.7266  Val Loss: 0.6841
Epoch 21  Train Loss: 0.7235  Val Loss: 0.6762
Epoch 22  Tra